In [ ]:
import urllib
import polars as pl
from zipfile import ZipFile

In [ ]:
url = "https://www.cs.utah.edu/~lifeifei/research/tpq/NA.cnode"

In [ ]:
na_road_network_df = pl.read_csv(url, has_header = False, sep = ' ')

In [ ]:
len(na_road_network_df)

In [ ]:
na_road_network_df.describe()

In [ ]:
# notice these are normalized, still don't know what that means and how to get them back into normal values for lattitude/longitude
na_road_network_df.head()

In [ ]:
len(na_road_network_df)

In [ ]:
cal_nodes_df = pl.read_csv("https://www.cs.utah.edu/~lifeifei/research/tpq/cal.cnode", has_header = False, sep = " ")
cal_edges_df = pl.read_csv("https://www.cs.utah.edu/~lifeifei/research/tpq/cal.cedge", has_header = False, sep = " ")

In [ ]:
len(cal_nodes_df), len(cal_edges_df)

In [ ]:
cal_nodes_df.head()

In [ ]:
# still don't know unit of the column 4, which is the l2 distance
cal_edges_df.head()

In [ ]:
trip_df = pl.read_csv("/home/therealchainman/Downloads/gps_data.csv")
road_network_df = pl.read_csv("/home/therealchainman/Downloads/road_network.csv")
gt_df = pl.read_csv("/home/therealchainman/Downloads/ground_truth_route.csv")

In [ ]:
trip_df.head()

In [ ]:
len(trip_df)

In [ ]:
trip_df.describe()

In [ ]:
trip_df

In [ ]:
road_network_df

In [ ]:
gt_df

In [ ]:

zip_file = "/home/therealchainman/Downloads/map-matching-dataset.zip"

In [ ]:
# create empty polars dataframe
"""
route table: contains the 0-indexed row number referring to an arc in the arcs table.  Ordered sequence of arcs on which the vehicle traveled.
track table: contains the longitude, latitude, and timestamp of each GPS point for a given vehicle trip.
nodes table: contains the longitude and latitude of each node in the road network.
arcs table: contains the 0-indexed row number of the source node and target node for each arc in the road network.
"""
# route_df = pl.DataFrame(columns = ['edge_id', 'id'])
# route_df.with_columns([pl.col('edge_id').cast(pl.Int64), pl.col('id').cast(pl.Int64)])
# track_df = pl.DataFrame(columns = ['long', 'lat', 'timestamp', 'id'])
# track_df.with_column(pl.col('id').cast(pl.Int64))
# nodes_df = pl.DataFrame(columns = ['long', 'lat', 'id'])
# nodes_df.with_column(pl.col('id').cast(pl.Int64))
# arcs_df = pl.DataFrame(columns = ['source_node', 'target_node', 'id'])
# arcs_df.with_columns([pl.col('source_node').cast(pl.Int64), pl.col('target_node').cast(pl.Int64), pl.col('id').cast(pl.Int64)])
route_df, track_df, nodes_df, arcs_df = pl.DataFrame(), pl.DataFrame(), pl.DataFrame(), pl.DataFrame()
for content in ZipFile(zip_file).namelist():
    try:
        id = int(content.split("/")[0])
    except ValueError:
        continue
    if content.endswith(".route"):
        cur_df = pl.read_csv(ZipFile(zip_file).read(content), has_header = False, sep = '\t')
        cur_df.columns = ['edge_id']
        cur_df = cur_df.with_column(pl.lit(id).alias("id"))
        if route_df.is_empty():
            route_df = cur_df
        else:
            route_df.extend(cur_df)
    elif content.endswith(".track"):
        cur_df = pl.read_csv(ZipFile(zip_file).read(content), has_header = False, sep = '\t')
        cur_df.columns = ['long', 'lat', 'timestamp']
        cur_df = cur_df.with_column(pl.lit(id).alias("id"))
        if track_df.is_empty():
            track_df = cur_df
        else:
            track_df.extend(cur_df)
    elif content.endswith(".nodes"):
        cur_df = pl.read_csv(ZipFile(zip_file).read(content), has_header = False, sep = '\t')
        cur_df.columns = ['long', 'lat']
        cur_df = cur_df.with_column(pl.lit(id).alias("id"))
        if nodes_df.is_empty():
            nodes_df = cur_df
        else:
            nodes_df.extend(cur_df)
    elif content.endswith(".arcs"):
        cur_df = pl.read_csv(ZipFile(zip_file).read(content), has_header = False, sep = '\t')
        cur_df.columns = ['source_node_id', 'target_node_id']
        cur_df = cur_df.with_column(pl.lit(id).alias("id"))
        if arcs_df.is_empty():
            arcs_df = cur_df
        else:
            arcs_df.extend(cur_df)

In [ ]:
len(route_df), len(track_df), len(nodes_df), len(arcs_df)

In [ ]:
nodes_df